In [2]:
import nltk
import numpy as np
import random
import string

# 1 DEFINICIÓN CORPUS

In [3]:
f=open(r'corpus_muse.txt','r',errors = 'ignore')
raw=f.read()
raw

'Muse es una plataforma web que permite a museos, empresas enfocadas al desarrollo cultural, instituciones y artistas independientes conectar con mÃ¡s pÃºblico, pero no solo eso, permite a los turistas, locales y amantes del arte conocer a una ciudad desde una perspectiva completamente cultural. Se trata de un movimiento de transformaciÃ³n hacia los cimientos de una ciudad inteligente que difunde su cultura y apoya a sus artistas.\n\nEn Tepic, Nayarit hay 6 museos que puedes visitar. \n\nEl Museo Interactivo de Ciencias e InnovaciÃ³n de Nayarit estÃ¡ ubicado en Av de la Salud, Ciudad Industrial y estÃ¡ abierto al pÃºblico de martes a domingo de 9 a 18 hrs.\n\nEl Museo Regional de Nayarit estÃ¡ ubicado en Av MÃ©xico Nte 91, Centro y estÃ¡ abierto al pÃºblico de lunes a viernes de 10 a 16 hrs SÃ¡bados de 10 a 15 hrs. \n\nLa entrada del Museo Regional de Nayarit tiene un costo de $65 mxn.\n\nLa Casa Museo Juan Escutia estÃ¡ ubicado en Miguel Hidalgo 71, Centro y estÃ¡ abierto de martes a 

# 2a Preprocesamiento del Texto con NTLK CORPUS

In [4]:
raw=raw.lower()# convertir en minúscula
#nltk.download('punkt') # Instalar módulo punkt si no está ya instalado (solo ejecutar la primera vez)
#nltk.download('wordnet') # Instalar módulo wordnet si no está ya instalado (solo ejecutar la primera vez)
sent_tokens = nltk.sent_tokenize(raw)# Convierte el CORPUS a una lista de sentencias
word_tokens = nltk.word_tokenize(raw)# Convierte el CORPUS a una lista de palabras
lemmer = nltk.stem.WordNetLemmatizer()

#WordNet diccionario semántico incluido en NLTK
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

# 2b PREPROCESAMIENTO DEL TEXTO + 3 Evaluar Similitud MENSAJE USUARIO - CORPUS

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords

#Función para determinar la similitud del texto insertado y el corpus
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response) #Añade al corpus la respuesta de usuario al final
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words=stopwords.words('spanish'))
    tfidf = TfidfVec.fit_transform(sent_tokens)
    # 3 EVALUAR SIMILITUD DE COSENO ENTRE MENSAJE USUARIO (tfidf[-1]) y el CORPUS (tfidf)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    
    if(req_tfidf==0):
        robo_response=robo_response+"Lo siento, no te he entendido. Si no puedo responder a lo que busca póngase en contacto con soportemuse@soporte.com"
        return robo_response

    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

# 4 DEFINICIÓN DE COINCIDENCIAS MANUAL

In [6]:
SALUDOS_INPUTS = ("hola", "buenas", "saludos", "qué tal", "hey","buenos dias","bonjour","coucou",)
SALUDOS_OUTPUTS = ["Hola", "Hola, ¿Qué tal?", "Hola, ¿Cómo te puedo ayudar?", "Hola, encantado de hablar contigo","Bonjour !","Coucou mon ami !"]

def saludos(sentence):
    for word in sentence.split():
        if word.lower() in SALUDOS_INPUTS:
            return random.choice(SALUDOS_OUTPUTS)

# 5 GENERACIÓN DE RESPUESTA

In [7]:
flag=True
print("MUSEBOT: Hola! Mi nombre es MuseBot, contestaré tus preguntas sobre esta plataforma y las opciones culturales de la ciudad de Tepic. Si quieres salir, escribe 'salir' ")
while(flag==True):
    user_response = input()
    user_response = user_response.lower() #Convertimos a minúscula
    
    if(user_response!='salir'):
        
        if(user_response=='gracias' or user_response=='muchas gracias'): #Se podría haber definido otra función de coincidencia manual
            flag=True
            print("MUSEBOT: De nada!")
            
        else:
            if(saludos(user_response)!=None): #Si la palabra insertada por el usuario es un saludo (Coincidencias manuales definidas previamente)
                print("MUSEBOT: "+saludos(user_response))
                
            else: #Si la palabra insertada no es un saludo --> CORPUS
                print("MUSEBOT: ",end="") 
                print(response(user_response))
                sent_tokens.remove(user_response) # para eliminar del corpus la respuesta del usuario y volver a evaluar con el CORPUS limpio
    else:
        flag=False
        print("MUSEBOT: Hasta pronto amigo, nos escribimos pronto!")

MUSEBOT: Hola! Mi nombre es MuseBot, contestaré tus preguntas sobre esta plataforma y las opciones culturales de la ciudad de Tepic. Si quieres salir, escribe 'salir' 
MUSEBOT: Hola, encantado de hablar contigo
MUSEBOT: Hasta pronto amigo, nos escribimos pronto!
